In [5]:
import pandas as pd
import numpy as np
import random
import nltk
import collections
import sklearn as sk

In [6]:
def data_cleaning(raw_data):
    '''
    This function takes in a string specifically "cyberpunk" and delete it.
    '''
    useful_words = raw_data.lower().split()
    useful_words = [w.replace('cyberpunk', '') for w in useful_words]
    return( " ".join(useful_words))

In [7]:
df = pd.read_csv("../../B_Data_Cleaning/cleaned_real_reviews.csv")
df["Review"] = df["Review"].astype("str")
df['Review'] = df['Review'].apply(data_cleaning)
df["Review"] = df["Review"].astype("str")

# make a set_column to count the number of words in each review
df["set_column"] = df["Review"].apply(lambda x: set(x.split()))
df

# split the data into train and test
train, test = sk.model_selection.train_test_split(df, test_size=0.33, random_state=42)

# # divide the df_cleaned based on "Recommended or Not Recommended" column
# temp_recommended = temp.loc[temp["Recommended or Not Recommended"] == True,:].reset_index(drop=True)
# temp_not_recommended = temp.loc[temp["Recommended or Not Recommended"] == False,:].reset_index(drop=True)

# # divide temp_recommended into train and test using random sample seed
# temp_recommended_train = temp_recommended.sample(frac=0.67, random_state=42).reset_index(drop=True)
# temp_recommended_test = temp_recommended.drop(temp_recommended_train.index).reset_index(drop=True)

# # divide temp_not_recommended into train and test using random sample seed
# temp_not_recommended_train = temp_not_recommended.sample(frac=0.33, random_state=42).reset_index(drop=True)
# temp_not_recommended_test = temp_not_recommended.drop(temp_not_recommended_train.index).reset_index(drop=True)

# # Combine test dataset
# temp_test = pd.concat([temp_recommended_test, temp_not_recommended_test], axis=0).reset_index(drop=True)

In [8]:
# Dividing dataset into recommended and not recommended
df_recom = train.loc[train["Recommended or Not Recommended"] == True,:]
df_recom = df_recom.reset_index(drop=True)

df_not_recom = train.loc[train["Recommended or Not Recommended"] == False,:]
df_not_recom = df_not_recom.reset_index(drop=True)

In [9]:
# This is to make the set dictionary
# count how many times each word appears in the reviews
bow_recom_temp = collections.Counter([y for x in df_recom.set_column for y in x])
bow_not_recom_temp = collections.Counter([y for x in df_not_recom.set_column for y in x])

bow_recom_set = dict(bow_recom_temp)
bow_not_recom_set = dict(bow_not_recom_temp)

for key in bow_recom_set:
    if key not in bow_not_recom_set:
        bow_not_recom_set[key] = 0
        pass
    else:
        pass

for key in bow_not_recom_set:
    if key not in bow_recom_set:
        bow_recom_set[key] = 0
        pass
    else:
        pass

for key in bow_recom_set:
    bow_recom_set[key] += 1

for key in bow_not_recom_set:
    bow_not_recom_set[key] += 1


In [10]:
# This is to make the list dictionary
# count how many times each word appears in the reviews
bow_recom = collections.Counter([y for x in df_recom.Review for y in x.split()])
bow_not_recom = collections.Counter([y for x in df_not_recom.Review for y in x.split()])

bow_recom_dict = dict(bow_recom)
bow_not_recom_dict = dict(bow_not_recom)

for key in bow_recom_dict:
    if key not in bow_not_recom_dict:
        bow_not_recom_dict[key] = 0
        pass
    else:
        pass

for key in bow_not_recom_dict:
    if key not in bow_recom_dict:
        bow_recom_dict[key] = 0
        pass
    else:
        pass

for key in bow_recom_dict:
    bow_recom_dict[key] += 1

for key in bow_not_recom_dict:
    bow_not_recom_dict[key] += 1
    

In [11]:
bow_recom_sum_vals = sum(bow_recom_dict.values())
bow_not_recom_sum_vals = sum(bow_not_recom_dict.values())

for i in bow_recom_dict:

    bow_recom_dict[i] /= bow_recom_sum_vals

for i in bow_not_recom_dict:

    bow_not_recom_dict[i] /= bow_not_recom_sum_vals

In [12]:
assert len(bow_recom_dict) == len(bow_not_recom_dict)
assert len(bow_recom_set) == len(bow_not_recom_set)

In [13]:
test.drop(columns=["set_column"], inplace=True)

In [14]:
def define_sentiment(element, test_all=False, test_only_tf_abs=False, test_only_idf_abs=False, test_only_prob=False, test_all_abs=False):

    """This function takes a review and returns the label for that review"""

    full_shape = df.shape[0]
    positive_review_probabiliy = len(df_recom)/full_shape
    negative_review_probabiliy = len(df_not_recom)/full_shape
    
    df_choices_positive = [positive_review_probabiliy, bow_recom_set, bow_recom_dict, df_recom]
    df_choices_negative = [negative_review_probabiliy ,bow_not_recom_set, bow_not_recom_dict, df_not_recom]

    def classifier(element, your_class = 'positive'):
        flag = {}
        flag_count = 0
        if your_class == 'positive':
            df_choices = df_choices_positive
        else:
            df_choices = df_choices_negative
        prob_of_class = df_choices[0]/full_shape
        score = 1 * prob_of_class
        score = float(format(score, '.12f'))
        # score = 0.1
        for i in element.split():
            if i not in df_choices[2].keys():
                pass
            else:
                prob_word_given_class = (df_choices[2])[i]
                prob_word_given_class = float(format(prob_word_given_class, '.12f'))
                # Almost the same value, given our spin on this application. 
                # Normally, this term frequency would be calculated differently across the positive and negative documents
                # but we are only looking at the reviews as the documents themselves to determine a word's relevance in the positive
                # or negative corpus. 
                tf = np.log(prob_word_given_class)
                # tf = float(format(tf, '.12f'))
                # tf = abs(np.log(prob_word_given_class))
                # The IDF is the number of reviews / the number of reviews that contain the word in that given corpus
                # idf = abs(np.log(df_choices[3].shape[0]/(df_choices[1])[i]))
                idf = np.log(df_choices[3].shape[0]/(df_choices[1])[i])
                # idf = float(format(idf, '.12f'))
                if test_all:
                    score *= prob_word_given_class*tf*idf
                elif test_only_tf_abs:
                    score *= prob_word_given_class*tf
                    score = abs(score)
                elif test_only_idf_abs:
                    score *= prob_word_given_class*idf
                    score = abs(score)
                elif test_only_prob:
                    score *= prob_word_given_class
                elif test_all_abs:
                    score *= prob_word_given_class*tf*idf
                    score = abs(score)

                if score < 0:
                    flag[flag_count] = (score, your_class)
                    # print(flag_count,i)
                    flag_count += 1
                    # return 'Flag'


        return score
    positive_score = classifier(element, 'positive')
    negative_score = classifier(element, 'negative')
    if positive_score > negative_score:
        return True
    elif positive_score == negative_score:
        # Choosing an arbitrary value, because we assume that a review with one or few words of little substance
        # is implied to be negative, as is usual with netizens. 
        return False
    else:
        return False

In [15]:
test["score_all"] = test.Review.apply(define_sentiment, test_all=True)
test["score_tf"] = test.Review.apply(define_sentiment, test_only_tf_abs=True)
test["score_idf"] = test.Review.apply(define_sentiment, test_only_idf_abs=True)
test["score_freq"] = test.Review.apply(define_sentiment, test_only_prob=True)
test["score_mod_all"] = test.Review.apply(define_sentiment, test_all_abs=True)

In [16]:
print("This is the result without the word 'cyberpunk' included in the corpus:")
print(f"This is our accuracy for (unbalanced) real data with all the formula: {(sum(test['Recommended or Not Recommended'] == test['score_all'])/test.shape[0])*100:.2f}%")
print(f"This is our accuracy for (unbalanced) real data with only class probability, frequency, and absolute value of TF: {(sum(test['Recommended or Not Recommended'] == test['score_tf'])/test.shape[0])*100:.2f}%")
print(f"This is our accuracy for (unbalanced) real data with only class probability, frequency, and absolute value of IDF: {(sum(test['Recommended or Not Recommended'] == test['score_idf'])/test.shape[0])*100:.2f}%")
print(f"This is our accuracy for (unbalanced) real data with only class probability and frequency: {(sum(test['Recommended or Not Recommended'] == test['score_freq'])/test.shape[0])*100:.2f}%")
print(f"This is our accuracy for (unbalanced) real data with all the formula, but with the absolute value of the score: {(sum(test['Recommended or Not Recommended'] == test['score_mod_all'])/test.shape[0])*100:.2f}%")

This is the result without the word 'cyberpunk' included in the corpus:
This is our accuracy for (unbalanced) real data with all the formula: 49.71%
This is our accuracy for (unbalanced) real data with only class probability, frequency, and absolute value of TF: 77.22%
This is our accuracy for (unbalanced) real data with only class probability, frequency, and absolute value of IDF: 69.55%
This is our accuracy for (unbalanced) real data with only class probability and frequency: 75.03%
This is our accuracy for (unbalanced) real data with all the formula, but with the absolute value of the score: 69.77%


In [17]:
# Generate a confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(test['Recommended or Not Recommended'], test['score_all'])
# Generate a classification report
from sklearn.metrics import classification_report
print(classification_report(test['Recommended or Not Recommended'], test['score_all']))


              precision    recall  f1-score   support

       False       0.35      0.58      0.44      1820
        True       0.69      0.45      0.55      3658

    accuracy                           0.50      5478
   macro avg       0.52      0.52      0.49      5478
weighted avg       0.57      0.50      0.51      5478



In [18]:
# Generate a confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(test['Recommended or Not Recommended'], test['score_tf'])
# Generate a classification report
from sklearn.metrics import classification_report
print(classification_report(test['Recommended or Not Recommended'], test['score_tf']))


              precision    recall  f1-score   support

       False       0.66      0.66      0.66      1820
        True       0.83      0.83      0.83      3658

    accuracy                           0.77      5478
   macro avg       0.74      0.74      0.74      5478
weighted avg       0.77      0.77      0.77      5478



In [19]:
# Generate a confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(test['Recommended or Not Recommended'], test['score_idf'])
# Generate a classification report
from sklearn.metrics import classification_report
print(classification_report(test['Recommended or Not Recommended'], test['score_idf']))


              precision    recall  f1-score   support

       False       0.56      0.38      0.45      1820
        True       0.73      0.85      0.79      3658

    accuracy                           0.70      5478
   macro avg       0.65      0.62      0.62      5478
weighted avg       0.68      0.70      0.68      5478



In [20]:
# Generate a confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(test['Recommended or Not Recommended'], test['score_freq'])
# Generate a classification report
from sklearn.metrics import classification_report
print(classification_report(test['Recommended or Not Recommended'], test['score_freq']))


              precision    recall  f1-score   support

       False       0.61      0.68      0.64      1820
        True       0.83      0.79      0.81      3658

    accuracy                           0.75      5478
   macro avg       0.72      0.73      0.73      5478
weighted avg       0.76      0.75      0.75      5478



Bad pipe message: %s [b'x\x1aD!;H\x08\xd6b\x05x\xe13\xbc\xbb\xf0\x04\xa9 \x91\xae]5\xfd\xfa\x11(`05q\xcc\xc8\xa51s\x91\n1+}`\xa1\xf3\xec+\x19\xeeg\x92\xa9\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00\x1c\x04\x03\x05\x03\x06\x03\x08\x07\x08\x08\x08\t\x08\n\x08\x0b\x08\x04\x08\x05\x08\x06\x04\x01\x05\x01\x06\x01\x00+\x00\x03\x02\x03\x04\x00-\x00\x02\x01\x01\x003\x00&\x00$\x00']
Bad pipe message: %s [b' 0\xa1[F\xcb\xa9\x19"\xc3\x18\xf3\xe3\x8cB\x96\xcf\xbfC\r\xf9o\x13\xa0\x9b\xa4!\x18']
Bad pipe message: %s [b"\xd0\x9a\x90\x1f\x18;\x94\x88\x10\xefe`m\xab\x14\xde>\x96 \x84\xb4\x93\xa9z\xb0\x80\xc41\xad\xb6^ \x03\\\xb4\xec\x89\xa8\xe7F0'D\x18~-B\x07[\x924\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x